<a href="https://colab.research.google.com/github/flying-bear/kompluxternaya/blob/master/assignment_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment 4: Named entity recognition

Create a model for Named Entity Recognition for dataset CoNLL 2002.  
Your quality metric = f1_macro

In your solution you should use: RandomForest, Gradient Boosting (xgboost, lightgbm, catboost)   
Tutorials:  
1. https://github.com/Microsoft/LightGBM/tree/master/examples/python-guide
1. https://github.com/catboost/tutorials 

More baselines you beat - better your score
 
baseline 1 [3 points]: 0.0604      random labels  
baseline 2 [5 points]: 0.3966      PoS features + logistic regression  
baseline 3 [8 points]: 0.8122      word2vec cbow embedding + baseline 2 + svm    

[1 point] using feature engineering (creating features not presented in the baselines)

! Your results must be reproducible. You should explicitly set all seeds random_states in yout model.  
! Remember to use proper training pipeline.  

bonus, think about:  
1. [1 point] Why did we select f1 score with macro averaging as our classification quality measure? What other metrics are suitable?   

In [0]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn import model_selection
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegressionCV
from sklearn.preprocessing import LabelEncoder
from sklearn import metrics

import warnings
warnings.filterwarnings('ignore')

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline


SEED=1337

In [34]:
from google.colab import drive
import os.path
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
root_path = 'gdrive/My Drive/studies/HSE/prog/kompluxternaya'

In [36]:
df = pd.read_csv(root_path + '/' + 'ner_short.csv', index_col=0)
df.head()

,next-next-pos,next-next-word,next-pos,next-word,pos,prev-pos,prev-prev-pos,prev-prev-word,prev-word,sentence_idx,word,tag
0,NNS,demonstrators,IN,of,NNS,__START1__,__START2__,__START2__,__START1__,1.0,Thousands,O
1,VBP,have,NNS,demonstrators,IN,NNS,__START1__,__START1__,Thousands,1.0,of,O
2,VBN,marched,VBP,have,NNS,IN,NNS,Thousands,of,1.0,demonstrators,O
3,IN,through,VBN,marched,VBP,NNS,IN,of,demonstrators,1.0,have,O
4,NNP,London,IN,through,VBN,VBP,NNS,demonstrators,have,1.0,marched,O


In [37]:
# number of sentences
df.sentence_idx.max()

1500.0

In [38]:
# class distribution
class_distr = df.tag.value_counts(normalize=True)
num_classes = len(class_distr)
print(num_classes)
class_distr

17


O        0.852828
B-geo    0.027604
B-gpe    0.020935
B-org    0.020247
I-per    0.017795
B-tim    0.016927
B-per    0.015312
I-org    0.013937
I-geo    0.005383
I-tim    0.004247
B-art    0.001376
I-gpe    0.000837
I-art    0.000748
B-eve    0.000628
I-eve    0.000508
B-nat    0.000449
I-nat    0.000239
Name: tag, dtype: float64

Как видно, соотношение классов несбалансированное: отсутствие именнованной сущности является наиболее частотным классом. Но данные по какому-то типу именновынных сущностей нам как минимум настолько же, если не более, интересны, чем данные по классу отсутствие-именнованной сущности. Таким образом, micro-averaging не подошёл бы, ведь он бы дал классам вес пропорциональные их представленности в выборке. Можно использовать любую macro-averaging метрику или weighted метрику или AUC.


In [0]:
# sentence length
tdf = df.set_index('sentence_idx')
tdf['length'] = df.groupby('sentence_idx').tag.count()
df = tdf.reset_index(drop=False)

In [0]:
# encode categorial variables

le = LabelEncoder()
df['pos'] = le.fit_transform(df.pos)
df['next-pos'] = le.fit_transform(df['next-pos'])
df['next-next-pos'] = le.fit_transform(df['next-next-pos'])
df['prev-pos'] = le.fit_transform(df['prev-pos'])
df['prev-prev-pos'] = le.fit_transform(df['prev-prev-pos'])

Давайте добавим фичу, которая считает число именованных сущностей в данном предложении.

In [0]:
# get number of named entities in a sentence (B markes the first word of a named entity)
def count_named(df):
  df['n_named_in_sent'] = len(df[df['tag'].apply(lambda s: s.startswith('B'))])
  return df

In [44]:
ex = df[df['sentence_idx'] == 1.0]
count_named(ex).head()

,sentence_idx,next-next-pos,next-next-word,next-pos,next-word,pos,prev-pos,prev-prev-pos,prev-prev-word,prev-word,word,tag,length,n_named_in_sent
0,1.0,18,demonstrators,9,of,18,39,40,__START2__,__START1__,Thousands,O,48,6
1,1.0,33,have,18,demonstrators,9,18,39,__START1__,Thousands,of,O,48,6
2,1.0,32,marched,33,have,18,9,18,Thousands,of,demonstrators,O,48,6
3,1.0,9,through,32,marched,33,18,9,of,demonstrators,have,O,48,6
4,1.0,16,London,9,through,32,33,18,demonstrators,have,marched,O,48,6


In [0]:
df = df.groupby('sentence_idx').apply(count_named)

In [47]:
df.head()

,sentence_idx,next-next-pos,next-next-word,next-pos,next-word,pos,prev-pos,prev-prev-pos,prev-prev-word,prev-word,word,tag,length,n_named_in_sent
0,1.0,18,demonstrators,9,of,18,39,40,__START2__,__START1__,Thousands,O,48,6
1,1.0,33,have,18,demonstrators,9,18,39,__START1__,Thousands,of,O,48,6
2,1.0,32,marched,33,have,18,9,18,Thousands,of,demonstrators,O,48,6
3,1.0,9,through,32,marched,33,18,9,of,demonstrators,have,O,48,6
4,1.0,16,London,9,through,32,33,18,demonstrators,have,marched,O,48,6


In [48]:
# splitting
y = LabelEncoder().fit_transform(df.tag)

df_train, df_test, y_train, y_test = model_selection.train_test_split(df, y, stratify=y, 
                                                                      test_size=0.25, random_state=SEED, shuffle=True)
print('train', df_train.shape[0])
print('test', df_test.shape[0])

train 50155
test 16719


In [0]:
# some wrappers to work with word2vec
from gensim.models.word2vec import Word2Vec
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.base import TransformerMixin
from collections import defaultdict

   
class Word2VecWrapper(TransformerMixin):
    def __init__(self, window=5,negative=5, size=100, iter=100, is_cbow=False, random_state=SEED):
        self.window_ = window
        self.negative_ = negative
        self.size_ = size
        self.iter_ = iter
        self.is_cbow_ = is_cbow
        self.w2v = None
        self.random_state = random_state
        
    def get_size(self):
        return self.size_

    def fit(self, X, y=None):
        """
        X: list of strings
        """
        sentences_list = [x.split() for x in X]
        self.w2v = Word2Vec(sentences_list, 
                            window=self.window_,
                            negative=self.negative_, 
                            size=self.size_, 
                            iter=self.iter_,
                            sg=not self.is_cbow_, seed=self.random_state)

        return self
    
    def has(self, word):
        return word in self.w2v

    def transform(self, X):
        """
        X: a list of words
        """
        if self.w2v is None:
            raise Exception('model not fitted')
        return np.array([self.w2v[w] if w in self.w2v else np.zeros(self.size_) for w in X ])
    


In [50]:
%%time
# here we exploit that word2vec is an unsupervised learning algorithm
# so we can train it on the whole dataset (subject to discussion)

sentences_list = [x.strip() for x in ' '.join(df.word).split('.')]

w2v_cbow = Word2VecWrapper(window=5, negative=5, size=300, iter=300, is_cbow=True, random_state=SEED)
w2v_cbow.fit(sentences_list)

CPU times: user 46 s, sys: 410 ms, total: 46.4 s
Wall time: 25.9 s


In [51]:
%%time
# baseline 1 
# random labels
from sklearn.preprocessing import OneHotEncoder
from sklearn.dummy import DummyClassifier


columns = ['pos','next-pos','next-next-pos','prev-pos','prev-prev-pos']

model = Pipeline([
    ('enc', OneHotEncoder()),
    ('est', DummyClassifier(random_state=SEED)),
])

model.fit(df_train[columns], y_train)

print('train', metrics.f1_score(y_train, model.predict(df_train[columns]), average='macro'))
print('test', metrics.f1_score(y_test, model.predict(df_test[columns]), average='macro'))


train 0.05887736725599869
test 0.060439542712750365
CPU times: user 142 ms, sys: 3.96 ms, total: 146 ms
Wall time: 150 ms


In [14]:
%%time
# baseline 2 
# pos features + one hot encoding + logistic regression
from sklearn.preprocessing import OneHotEncoder


columns = ['pos','next-pos','next-next-pos','prev-pos','prev-prev-pos']

model = Pipeline([
    ('enc', OneHotEncoder()),
    ('est', LogisticRegressionCV(Cs=5, cv=5, n_jobs=-1, scoring='f1_macro', 
                             penalty='l2', solver='newton-cg', multi_class='multinomial', random_state=SEED)),
])

model.fit(df_train[columns], y_train)

print('train', metrics.f1_score(y_train, model.predict(df_train[columns]), average='macro'))
print('test', metrics.f1_score(y_test, model.predict(df_test[columns]), average='macro'))

train 0.46639500282346874
test 0.39660981421559566
CPU times: user 4min 13s, sys: 1.38 s, total: 4min 14s
Wall time: 22min 35s


In [52]:
## pos features + one hot encoding + grid search CV on random forest
%%time
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestClassifier


columns = ['pos','next-pos','next-next-pos','prev-pos','prev-prev-pos']

model = Pipeline([
    ('enc', OneHotEncoder()),
    ('est', model_selection.GridSearchCV(RandomForestClassifier(random_state=SEED), 
                                         {'n_estimators': [10, 45, 48, 50]}, 
                                         cv=3, scoring='f1_macro', n_jobs=-1, verbose=1)),
])

model.fit(df_train[columns], y_train)
print(model['est'].best_params_)


print('train', metrics.f1_score(y_train, model.predict(df_train[columns]), average='macro'))
print('test', metrics.f1_score(y_test, model.predict(df_test[columns]), average='macro'))


Fitting 3 folds for each of 4 candidates, totalling 12 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  12 out of  12 | elapsed:   46.4s finished


{'n_estimators': 48}
train 0.7481422448080776
test 0.5975623437925786
CPU times: user 11 s, sys: 118 ms, total: 11.2 s
Wall time: 57.5 s


In [16]:
%%time
# baseline 3
# use word2vec cbow embedding + baseline 2 + svm
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.svm import LinearSVC
import scipy.sparse as sp

embeding = w2v_cbow
encoder_pos = OneHotEncoder()
X_train = sp.hstack([
    embeding.transform(df_train.word),
    embeding.transform(df_train['next-word']),
    embeding.transform(df_train['next-next-word']),
    embeding.transform(df_train['prev-word']),
    embeding.transform(df_train['prev-prev-word']),
    encoder_pos.fit_transform(df_train[['pos','next-pos','next-next-pos','prev-pos','prev-prev-pos']])
])
X_test = sp.hstack([
    embeding.transform(df_test.word),
    embeding.transform(df_test['next-word']),
    embeding.transform(df_test['next-next-word']),
    embeding.transform(df_test['prev-word']),
    embeding.transform(df_test['prev-prev-word']),
    encoder_pos.transform(df_test[['pos','next-pos','next-next-pos','prev-pos','prev-prev-pos']])
])

model = model_selection.GridSearchCV(LinearSVC(penalty='l2', multi_class='ovr', random_state=SEED), 
                                    {'C': np.logspace(-4, 0, 5)}, 
                                    cv=3, scoring='f1_macro', n_jobs=-1, verbose=1)
model.fit(X_train, y_train)
print(model.best_params_)

print('train', metrics.f1_score(y_train, model.predict(X_train), average='macro'))
print('test', metrics.f1_score(y_test, model.predict(X_test), average='macro'))

Fitting 3 folds for each of 5 candidates, totalling 15 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  15 out of  15 | elapsed: 13.3min finished


{'C': 0.1}
train 0.9554835304374556
test 0.7807413231353165
CPU times: user 2min 29s, sys: 1.37 s, total: 2min 31s
Wall time: 15min 49s


In [55]:
# word2vec cbow embedding + pos features + one hot encoding + random forest
from sklearn.ensemble import RandomForestClassifier

model = model_selection.GridSearchCV(RandomForestClassifier(random_state=SEED), 
                                    {'n_estimators': [10, 25, 48, 50]}, 
                                    cv=3, scoring='f1_macro', n_jobs=-1, verbose=1)
model.fit(X_train, y_train)
print(model.best_params_)


print('train', metrics.f1_score(y_train, model.predict(X_train), average='macro'))
print('test', metrics.f1_score(y_test, model.predict(X_test), average='macro'))

Fitting 3 folds for each of 4 candidates, totalling 12 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  12 out of  12 | elapsed: 18.9min finished


{'n_estimators': 50}
train 0.9939909190143295
test 0.8649591696976522


In [56]:
# word2vec cbow embedding + pos features + one hot encoding + lightGBM random forest
%%time
import lightgbm as lgb

params = {
    'boosting_type': 'rf',  # random forest right here
    'objective': 'multiclass',
    'num_leaves': 31,
    'learning_rate': 0.5,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'num_classes': num_classes,
    'verbose': 5,
    'num_boost_rounds': 50
}

lgb_train = lgb.Dataset(data = X_train.toarray(), label = y_train)
lgb_test = lgb.Dataset(X_test.toarray(), y_test, reference=lgb_train)

gbm = lgb.train(params,
                lgb_train,
                valid_sets=lgb_test,
                early_stopping_rounds=5)

[1]	valid_0's multi_logloss: 1.88772
Training until validation scores don't improve for 5 rounds.
[2]	valid_0's multi_logloss: 1.766
[3]	valid_0's multi_logloss: 1.68187
[4]	valid_0's multi_logloss: 1.73595
[5]	valid_0's multi_logloss: 1.74133
[6]	valid_0's multi_logloss: 1.72367
[7]	valid_0's multi_logloss: 1.69399
[8]	valid_0's multi_logloss: 1.68335
Early stopping, best iteration is:
[3]	valid_0's multi_logloss: 1.68187
CPU times: user 2min 4s, sys: 777 ms, total: 2min 5s
Wall time: 1min 7s


In [57]:
y_pred_train = gbm.predict(X_train, num_iteration=gbm.best_iteration)
y_pred_train = [np.argmax(line) for line in y_pred_train]
y_pred_test = gbm.predict(X_test, num_iteration=gbm.best_iteration)
y_pred_test = [np.argmax(line) for line in y_pred_test]

print('train', metrics.f1_score(y_train, y_pred_train, average='macro'))
print('test', metrics.f1_score(y_test, y_pred_test, average='macro'))

train 0.4747496400179456
test 0.42304080942587147


чтоб результаты lgbm были лучше нужно подбирать параметры, но модель дорогая и это запарно